In [4]:

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

emails = [
    "무료 쿠폰 지급 클릭하세요",
    "당첨되셨습니다 지금 수령하세요",
    "무료 상품권 당첨 축하합니다",
    "회의 일정 확인 부탁드립니다",
    "프로젝트 제출 마감일 안내",
    "내일 점심 약속 확인",
    "무료 체험단 모집 중입니다",
    "월간 보고서 검토 요청",
]

labels = [1,1,1,0,0,0,1,0]

X_train, X_test, y_train, y_test = train_test_split(
    emails, labels, test_size=0.25, random_state=42
)


In [5]:
# 한국어에서 강력한 기본 베이스라인 (형태소 없이도 성능 좋음)
vectorizer = CountVectorizer(analyzer="char", ngram_range=(2,4))

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("학습 데이터 shape:", X_train_vec.shape)



학습 데이터 shape: (6, 208)


In [6]:
model = MultinomialNB(alpha=1.0)  # 라플라스 스무딩


model.fit(X_train_vec, y_train)


preds = model.predict(X_test_vec)


print("정확도:", accuracy_score(y_test, preds))
print("혼동행렬:")
print(confusion_matrix(y_test, preds))


정확도: 1.0
혼동행렬:
[[1 0]
 [0 1]]


In [7]:
proba = model.predict_proba(X_test_vec)

for mail, p in zip(X_test, proba):
    print(f"메일: {mail}")
    print(f"  스팸 확률: {p[1]:.3f}")


메일: 당첨되셨습니다 지금 수령하세요
  스팸 확률: 0.977
메일: 내일 점심 약속 확인
  스팸 확률: 0.055


In [8]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate

In [9]:
id_ = "seowoong362@gmail.com"
pass_ = 'yciu epuq dsry iuzs'   

In [10]:
smtp = smtplib.SMTP_SSL("smtp.gmail.com", 465)
smtp.login(id_, pass_)

(235, b'2.7.0 Accepted')

In [64]:
email_from = id_
email_to = id_
email_date = formatdate(localtime=True)
email_subject = "영화 달콤한 인생 - 명대사"
email_message = """
안녕하세요. 

제가 좋아하는 영화의 좋아하는 대사를 적어봅니다. 

어느 맑은 봄날, 바람에 이리저리 휘날리는 나뭇가지를 바라보며 제자가 물었다.

스승님, 저것은 나뭇가지가 움직이는 겁니까, 바람이 움직이는 겁니까? 

스승은 제자가 가리키는 곳은 보지도 않은 채, 웃으며 말했다.

무릇, 움직이는 것은 나뭇가지도 아니고, 바람도 아니며, 네 마음뿐이다.
"""


In [14]:
msg = MIMEMultipart('alternative')

# 보내는 사람 
msg['From'] = email_from

# 받는 사람 
msg['To'] = email_to

# 날짜
msg['Date'] = email_date

# 제목
msg['Subject'] = email_subject


In [15]:
msg.attach(MIMEText(email_message))

In [16]:
smtp.sendmail(id_, email_to,  msg.as_string())

{}

In [17]:
import imaplib
mail = imaplib.IMAP4_SSL("imap.gmail.com", 993)

In [18]:
mail.login(id_, pass_)

('OK', [b'seowoong362@gmail.com authenticated (Success)'])

In [19]:
mail.select("inbox")

('OK', [b'7'])

In [54]:
status, messages = mail.search(None, "SEEN")

In [55]:
messages

[b'1 2 3 4 5 6 7']

In [56]:
status, msg_data = mail.fetch(messages[0].split()[5], "(RFC822)")

In [57]:
len(msg_data)

2

In [58]:
import email
msg = email.message_from_bytes(msg_data[0][1])

In [60]:
from email.header import decode_header
subject = decode_header(msg['Subject'])[0][0].decode('utf-8')

In [62]:
msg.get("From")

'seowoong362@gmail.com'

In [63]:
if msg.is_multipart():
    for part in msg.walk():
        content_type = part.get_content_type()
        content_disposition = str(part.get("Content-Disposition"))


        if content_type == "text/plain" and "attachment" not in content_disposition:
            body = part.get_payload(decode=True).decode("utf-8", errors="ignore")
            print(f"내용:\n{body[:200]}...")  # 앞 200자만 출력
            break
else:
    body = msg.get_payload(decode=True).decode("utf-8", errors="ignore")
    print(f"내용:\n{body[:200]}...")

내용:

안녕하세요. 

제가 좋아하는 영화의 좋아하는 대사를 적어봅니다. 

어느 맑은 봄날, 바람에 이리저리 휘날리는 나뭇가지를 바라보며 제자가 물었다.

스승님, 저것은 나뭇가지가 움직이는 겁니까, 바람이 움직이는 겁니까? 

스승은 제자가 가리키는 곳은 보지도 않은 채, 웃으며 말했다.

무릇, 움직이는 것은 나뭇가지도 아니고, 바람도 아니며, 네 마음뿐이...


In [68]:
import pandas as pd

In [66]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
corpus = [
    "동해물과 백두산이",
    "넌 나에게 모욕감을 줬어",
    "맥북 싸게 팔아요",
    "맥북 프로 가격 얼마인가요, 맥북 에어도 있어요"
]


X = vectorizer.fit_transform(corpus)

In [69]:
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

,가격,나에게,동해물과,맥북,모욕감을,백두산이,싸게,얼마인가요,에어도,있어요,줬어,팔아요,프로
0,0,0,1,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,0,0,0,1,0
3,1,0,0,2,0,0,0,1,1,1,0,0,1


In [71]:
df = pd.read_csv("./spam_ham_3000.csv")

In [76]:
df['text'] = df['subject'] + df['body']

In [74]:
x = df['text']
y = df['label']

In [75]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [77]:
vectorizer = CountVectorizer(
    max_features=5000,
)

In [79]:
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)


In [80]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=1.0)
model.fit(x_train_vec, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [81]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = model.predict(x_test_vec)

In [83]:
accuracy_score(y_test, y_pred)

1.0

In [89]:
data_vec = vectorizer.transform(["내일 오전 10시에 여의도에서 당첨 확률 100% 리딩방 오세요."])
model.predict_proba(data_vec)
model.predict(data_vec)

array(['spam'], dtype='<U4')

In [91]:
email_from = id_
email_to = id_
email_date = formatdate(localtime=True)
email_subject = "영화 달콤한 인생 - 명대사"
email_message = """
이벤트 당첨을 축하드립니다. 상품권 수령하기
"""


In [92]:
msg = MIMEMultipart('alternative')

# 보내는 사람 
msg['From'] = email_from

# 받는 사람 
msg['To'] = email_to

# 날짜
msg['Date'] = email_date

# 제목
msg['Subject'] = email_subject
